In [0]:
%sql
DROP TABLE IF EXISTS events;


In [0]:
%sql
SHOW VOLUMES IN workspace.ecommerce;


database,volume_name
ecommerce,delta
ecommerce,ecommerce_data


In [0]:
%sql
CREATE VOLUME workspace.ecommerce.events;


---------------------------------------------------------------------------
AnalysisException                         Traceback (most recent call last)
File <command-7657307189748126>, line 1
----> 1 get_ipython().run_cell_magic('sql', '', 'CREATE VOLUME workspace.ecommerce.events;\n')

File /databricks/python/lib/python3.12/site-packages/IPython/core/interactiveshell.py:2541, in InteractiveShell.run_cell_magic(self, magic_name, line, cell)
   2539 with self.builtin_trap:
   2540     args = (magic_arg_s, cell)
-> 2541     result = fn(*args, **kwargs)
   2543 # The code below prevents the output from being displayed
   2544 # when using magics with decorator @output_can_be_silenced
   2545 # when the last Python token in the expression is a ';'.
   2546 if getattr(fn, magic.MAGIC_OUTPUT_CAN_BE_SILENCED, False):

File /databricks/python_shell/lib/dbruntime/sql_magic/sql_magic.py:194, in SqlMagic.sql(self, line, cell)
    188 except BaseException as e:
    189     self.driver_activity_log

In [0]:
from delta.tables import DeltaTable
from pyspark.sql import functions as F

# -----------------------------
# Paths (Unity Catalog Volume)
# -----------------------------
csv_path = "/Volumes/workspace/ecommerce/ecommerce_data/2019-Oct.csv"
delta_path = "/Volumes/workspace/ecommerce/events/events_delta"

# -----------------------------
# Read source CSV
# -----------------------------
updates = spark.read.format("csv") \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .load(csv_path)

# -----------------------------
# Create Delta table if not exists
# -----------------------------
if not DeltaTable.isDeltaTable(spark, delta_path):
    updates.write.format("delta") \
        .mode("overwrite") \
        .save(delta_path)

# -----------------------------
# Load Delta table
# -----------------------------
delta_table = DeltaTable.forPath(spark, delta_path)

# -----------------------------
# MERGE (deterministic key)
# -----------------------------
delta_table.alias("t").merge(
    updates.alias("s"),
    """
    t.user_session = s.user_session
    AND t.event_time = s.event_time
    AND t.product_id = s.product_id
    """
).whenMatchedUpdateAll() \
 .whenNotMatchedInsertAll() \
 .execute()


---------------------------------------------------------------------------
UnsupportedOperationException             Traceback (most recent call last)
File <command-7657307189748124>, line 43
     29 delta_table = DeltaTable.forPath(spark, delta_path)
     31 # -----------------------------
     32 # MERGE (deterministic key)
     33 # -----------------------------
     34 delta_table.alias("t").merge(
     35     updates.alias("s"),
     36     """
     37     t.user_session = s.user_session
     38     AND t.event_time = s.event_time
     39     AND t.product_id = s.product_id
     40     """
     41 ).whenMatchedUpdateAll() \
     42  .whenNotMatchedInsertAll() \
---> 43  .execute()

File /databricks/python/lib/python3.12/site-packages/delta/connect/tables.py:609, in DeltaMergeBuilder.execute(self)
    599 plan = MergeIntoTable(
    600     self._target,
    601     self._source,
   (...)
    606     self._with_schema_evolution
    607 )
    608 df = DataFrame(plan, session=self._s